# Import dependancies

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import bz2

In [ ]:

bookings_path = r'C:\Users\Usuario\Desktop\Master DATA SCIENCE\00_github_notebooks\Notebooks Apuntes\data\challenge\bookings.csv.bz2'
searches_path = r'C:\Users\Usuario\Desktop\Master DATA SCIENCE\00_github_notebooks\Notebooks Apuntes\data\challenge\searches.csv.bz2'

zip_file = bz2.BZ2File(bookings_path, 'r')
df = zip_file.read()
df

In [7]:
with bz2.BZ2File(bookings_path) as f:
    # Decompress data from file
    content = f.read()

In [5]:
bookings_path = r'C:\Users\Usuario\Desktop\Master DATA SCIENCE\data\challenge\bookings.csv'
temp = pd.read_csv(bookings_path, sep='delimiter', header=None).head(10000)
temp.to_csv('sample.csv', index=False)

<ipython-input-5-b5baa44d6dc9>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  temp = pd.read_csv(bookings_path, sep='delimiter', header=None).head(10000)


In [2]:
#en linux para abrir el archivo
!bzcat ./challenge/bookings.csv.bz2 | head -10000 > bookings.sample.csv

"bzcat" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [96]:
bookings_path = r'./sample.csv.bz2'
df = pd.read_csv(bookings_path, compression='bz2', sep= '^', header=1, usecols=['arr_port', 'pax', 'year'], nrows = 100000)

In [97]:
df.columns

Index(['arr_port', 'pax', 'year'], dtype='object')

In [48]:
df.columns = [col.strip() for col in df.columns]

In [56]:
df.loc[:,'arr_port'] = df['arr_port'].str.strip()

In [52]:
new_cols = {}
for col in df.columns:
    new_cols[col] = col.strip()
df = df.rename(new_cols, axis=1)

In [121]:
bookings_path = r'C:/Users/Usuario/Desktop/Master DATA SCIENCE/00_github_notebooks/Notebooks Apuntes/data/challenge/bookings.csv.bz2'
df_big = pd.read_csv(
                     bookings_path,
                     compression='bz2', 
                     sep= '^', 
                     header=0, 
                     usecols=['arr_port', 'pax', 'year'], 
                     #nrows = 100000
                     chunksize=100_000 # con este parametro se convierte en un iterador
                    )

In [122]:
with pd.read_csv(
                     bookings_path,
                     compression='bz2', 
                     sep= '^', 
                     header=0, 
                     usecols=['arr_port', 'pax', 'year'], 
                     #nrows = 100000
                     chunksize=100_000 # con este parametro se convierte en un iterador
                    ) as df_iter:
                    for chunk in df_big:
                        print(len(chunk))
                        break


100000


In [124]:
df_iter = pd.read_csv(
                     bookings_path,
                     compression='bz2', 
                     sep= '^', 
                     header=0, 
                     usecols=['arr_port', 'pax', 'year'], 
                     nrows = 100_000
                     chunksize=10_000 # con este parametro se convierte en un iterador
                    ) 
for chunk in df_big:
    print(chunk.head(10))
    break


        arr_port  pax  year
200000  DFW         5  2013
200001  DFW        -5  2013
200002  DFW         5  2013
200003  DFW        -5  2013
200004  DFW         5  2013
200005  DFW         1  2013
200006  DFW         1  2013
200007  DFW        -1  2013
200008  PNH         1  2013
200009  PNH        -1  2013


In [106]:
df_big.loc[:,'arr_port'] = df_big['arr_port'].str.strip()


In [118]:
df_big.groupby('arr_port')['pax'].sum().reset_index().sort_values(by='pax', ascending=False).head(10)

,arr_port,pax
733,LHR,1006
798,MCO,861
620,JFK,795
706,LAX,761
156,BKK,747
704,LAS,732
1159,SFO,705
957,ORD,686
232,CDG,676
366,DXB,587


https://pypi.org/project/GeoBases/

In [ ]:
''' 
zip_file = bz2.open(searches_path)
csv_file_reference = zip_file.read(zip_file)
searches = pd.read_csv(csv_file_reference)
'''

# Explore the data available

In [29]:
zip_file

# Exercise 1

Count the number of lines in python for each file

# Exercise 2

Top 10 arrival airports in the world in 2013 (using the bookings file)

• Arrival airport is the column arr_port. It is the IATA code for the airport

• To get the total number of passengers for an airport, you can sum the column
pax, grouping by arr_port. Note that there is negative pax. That corresponds to
cancelations. So to get the total number of passengers that have actually
booked, you should sum including the negatives (that will remove the canceled
bookings).

• Print the top 10 arrival airports in the standard output, including the number of
passengers.

• Bonus point: Get the name of the city or airport corresponding to that airport
(programatically, we suggest to have a look at GeoBases in Github)

• Bonus point: Solve this problem using pandas (instead of any other approach)

# Exercise 3

Plot the monthly number of searches for flights arriving at Málaga, Madrid or Barcelona

• For the arriving airport, you can use the
Destination column in the searches file.

• Plot a curve for Málaga, another one for
Madrid, and another one for Barcelona, in
the same figure.

• Bonus point: Solving this problem using
pandas (instead of any other approach)

# Exercise 4
Match searches with bookings

• For every search in the searches file, find out whether
the search ended up in a booking or not (using the info
in the bookings file). For instance, search and booking
origin and destination should match.

• For the bookings file, origin and destination are the
columns dep_port and arr_port, respectively.

• Generate a CSV file with the search data, and an
additional field, containing 1 if the search ended up in
a booking, and 0 otherwise.

# Exercise 5

Write a Web Service

• Wrap the output of the second exercise in a
web service that returns the data in JSON
format (instead of printing to the standard
output).

• The web service should accept a parameter
n>0. For the top 10 airports, n is 10. For the X
top airports, n is X 